# How to load data to BigQuery

Here we demonstrate a few different ways to load data to BigQuery from an R notebook.
* [bq](https://cloud.google.com/bigquery/docs/bq-command-line-tool) command line tool
* [bigrquery](https://cloud.google.com/blog/products/gcp/google-cloud-platform-for-data-scientists-using-r-with-google-bigquery-part-2-storing-and-retrieving-data-frames)

## Setup

In [1]:
library(jsonlite)
library(bigrquery)
library(lubridate)
library(tidyverse)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.1     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date()        masks base::date()
✖ dplyr::filter()          masks stats::filter()
✖ purrr::flatten()         masks jsonlite::flatten()
✖ lubridate::intersect()   masks base::intersect()
✖ dplyr::lag()             masks stats::lag()
✖ lubridate::setdiff()     masks base::setdiff()
✖ lubridate::union()       masks base::union()



Edit these global variables in your clone of this notebook if you do not have permission to WRITE data to this native Google Cloud Platform project.
* The destination BigQuery dataset should already exist. Your pet account must have WRITE access to it.
* The remaining cells can be run as-is.

In [2]:
# CHANGE THESE VARIABLES, IF NEEDED
DESTINATION_PROJECT_ID <- 'terra-resources'
DESTINATION_DATASET <- 'autodelete_after_one_day'

In [3]:
# This file loads fine via autodetect.
CSV_PATH <- 'gs://genomics-public-data/platinum-genomes/other/platinum_genomes_sample_info.csv'

# Also try this CSV which will yield some autodetect errors.
CSV_PATH_AUTODETECT_FAILS <- 'gs://genomics-public-data/1000-genomes/other/sample_info/sample_info.csv'

BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

# Load data to BigQuery from a CSV

We'll do this using the `bq` command line tool and the `--autodetect` flag.

In [4]:
DESTINATION_TABLE <- paste0('r_bq_autodetect_', strftime(now(), '%Y%m%d_%H%M%S'))

In [5]:
system(str_glue(str_c('bq --project_id {BILLING_PROJECT_ID} load ',
                      '--autodetect ',
                      '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE} ',
                      '{CSV_PATH}  2>&1')),
      intern = TRUE)

[1] "\rWaiting on bqjob_r24af6b9001c9caf4_000001739230af63_1 ... (0s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r24af6b9001c9caf4_000001739230af63_1 ... (1s) Current status: RUNNING\r                                                                                      \rWaiting on bqjob_r24af6b9001c9caf4_000001739230af63_1 ... (1s) Current status: DONE   "

Show the table schema.

In [6]:
system(str_glue(str_c('bq --project_id {BILLING_PROJECT_ID} show ',
                      '{DESTINATION_PROJECT_ID}:{DESTINATION_DATASET}.{DESTINATION_TABLE}')),
      intern = TRUE)

[1] "Table terra-resources:autodelete_after_one_day.r_bq_autodetect_20200727_213210"                                                                    
 [2] ""                                                                                                                                                     
 [3] "   Last modified                  Schema                 Total Rows   Total Bytes     Expiration      Time Partitioning   Clustered Fields   Labels  "
 [4] " ----------------- ------------------------------------ ------------ ------------- ----------------- ------------------- ------------------ -------- "
 [5] "  27 Jul 21:32:13   |- Catalog_ID: string                17           1441          03 Aug 21:32:13                                                  "
 [6] "                    |- Description: string                                                                                                           "
 [7] "                    |- Gender: string                                                                                                                "
 [8] "                    |- Race: string                                                                                                                  "
 [9] "                    |- Family_Member: integer                                                                                                        "
[10] "                    |- Relationship_to_Proband: string                                                                                               "
[11] "                    |- CT_Desc: string                                                                                                               "
[12] "                    |- Cum_Pdl: string                                                                                                               "
[13] ""

# Load data to BigQuery from a dataframe

We'll do this using R package `bigrquery`.

In [7]:
DESTINATION_TABLE <- paste0('r_bigrquery_', strftime(now(), '%Y%m%d_%H%M%S'))

In [8]:
insert_upload_job(project = DESTINATION_PROJECT_ID,
                  dataset = DESTINATION_DATASET,
                  table = DESTINATION_TABLE,
                  billing = BILLING_PROJECT_ID,
                  write_disposition = 'WRITE_EMPTY',
                  mtcars)

Warning message:
“'insert_upload_job' is deprecated.
Use 'bq_perform_upload' instead.
See help("Deprecated") and help("bigrquery-deprecated").”


$kind
[1] "bigquery#job"

$etag
[1] "FT47PU2htsfGt10G0aWlfg=="

$id
[1] "fc-product-demo:US.job_osQH4nNCEWVG1zJTmeQIq13svqfV"

$selfLink
[1] "https://www.googleapis.com/bigquery/v2/projects/fc-product-demo/jobs/job_osQH4nNCEWVG1zJTmeQIq13svqfV?location=US"

$user_email
[1] "pet-110502112294304402890@fc-product-demo.iam.gserviceaccount.com"

$configuration
$configuration$load
$configuration$load$schema
$configuration$load$schema$fields
$configuration$load$schema$fields[[1]]
$configuration$load$schema$fields[[1]]$name
[1] "mpg"

$configuration$load$schema$fields[[1]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[1]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[1]]$description
[1] ""


$configuration$load$schema$fields[[2]]
$configuration$load$schema$fields[[2]]$name
[1] "cyl"

$configuration$load$schema$fields[[2]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[2]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[2]]$description
[1] ""


$configuration$load$schema$fields[[3]]
$configuration$load$schema$fields[[3]]$name
[1] "disp"

$configuration$load$schema$fields[[3]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[3]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[3]]$description
[1] ""


$configuration$load$schema$fields[[4]]
$configuration$load$schema$fields[[4]]$name
[1] "hp"

$configuration$load$schema$fields[[4]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[4]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[4]]$description
[1] ""


$configuration$load$schema$fields[[5]]
$configuration$load$schema$fields[[5]]$name
[1] "drat"

$configuration$load$schema$fields[[5]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[5]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[5]]$description
[1] ""


$configuration$load$schema$fields[[6]]
$configuration$load$schema$fields[[6]]$name
[1] "wt"

$configuration$load$schema$fields[[6]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[6]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[6]]$description
[1] ""


$configuration$load$schema$fields[[7]]
$configuration$load$schema$fields[[7]]$name
[1] "qsec"

$configuration$load$schema$fields[[7]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[7]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[7]]$description
[1] ""


$configuration$load$schema$fields[[8]]
$configuration$load$schema$fields[[8]]$name
[1] "vs"

$configuration$load$schema$fields[[8]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[8]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[8]]$description
[1] ""


$configuration$load$schema$fields[[9]]
$configuration$load$schema$fields[[9]]$name
[1] "am"

$configuration$load$schema$fields[[9]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[9]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[9]]$description
[1] ""


$configuration$load$schema$fields[[10]]
$configuration$load$schema$fields[[10]]$name
[1] "gear"

$configuration$load$schema$fields[[10]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[10]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[10]]$description
[1] ""


$configuration$load$schema$fields[[11]]
$configuration$load$schema$fields[[11]]$name
[1] "carb"

$configuration$load$schema$fields[[11]]$type
[1] "FLOAT"

$configuration$load$schema$fields[[11]]$mode
[1] "NULLABLE"

$configuration$load$schema$fields[[11]]$description
[1] ""




$configuration$load$destinationTable
$configuration$load$destinationTable$projectId
[1] "terra-resources"

$configuration$load$destinationTable$datasetId
[1] "autodelete_after_one_day"

$configuration$load$destinationTable$tableId
[1] "r_bigrquery_20200727_213214"


$configuration$load$createDisposition
[1] "CREATE_IF_NEEDED"

$configuration$load$writeDisposition
[1] "WRITE_EMPTY"

$configuration$load$sourceFormat
[1] "NEWLINE_DELIMITED_JSON"


$configuration$jobType
[1] "LOAD"


$jobReference
$jobReference$projectId
[1] "fc-product-demo"

$jobReference$jobId
[

In [10]:
# Create a "connection" to a public BigQuery dataset.
dbcon <- bigrquery::src_bigquery(project = DESTINATION_PROJECT_ID,
                                 dataset = DESTINATION_DATASET,
                                 billing = BILLING_PROJECT_ID)

# Create a 'virtual dataframe' backed by a BigQuery table.
tbl <- dplyr::tbl(dbcon, DESTINATION_TABLE)
colnames(tbl)

[1] "mpg"  "cyl"  "disp" "hp"   "drat" "wt"   "qsec" "vs"   "am"   "gear"
[11] "carb"

# Provenance

In [11]:
devtools::session_info()

─ Session info ───────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 4.0.2 (2020-06-22)
 os       Ubuntu 18.04.4 LTS          
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2020-07-27                  

─ Packages ───────────────────────────────────────────────────────────────────
 package     * version  date       lib source        
 assertthat    0.2.1    2019-03-21 [2] CRAN (R 4.0.2)
 backports     1.1.8    2020-06-17 [2] CRAN (R 4.0.2)
 base64enc     0.1-3    2015-07-28 [2] CRAN (R 4.0.2)
 bigrquery   * 1.3.1    2020-05-15 [2] CRAN (R 4.0.2)
 bit           1.1-15.2 2020-02-10 [2] CRAN (R 4.0.2)
 bit64         0.9-7    2017-05-08 [2] CRAN (R 4.0.2)
 blob          1.2.1    2020-01-20 [2] CRAN (R 4.0.2)
 broom         0.5.

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.